In [ ]:
# simulate portfolio performance
# calcualte realized irr; back of envelope calculation irr
# irr interpretation

In [70]:
import pandas as pd
import numpy as np
import sys
import numpy_financial as npf
sys.path.append('..') 
from AssetModeling.Asset import Asset
from Utils.SPCFUtils import SPCFUtils
import warnings
warnings.filterwarnings('ignore')

In [116]:
notionalStore = np.arange(5000, 50000, 300)
termStore = [36, 60 ,72]
intRateStore = np.arange(0.1, 0.3, 0.01)

cdrVectorStringStore = ["0 ramp 7 11",
                        "0 ramp 10 20 20 ramp 6 15",
                        "0 ramp 15 18",
                        "0 ramp 16 13 13 for 5 13 ramp 6 18"
                        ]

cprVectorStringStore = ["10 ramp 12 7",
                        "8",
                        "10"
                        ]

sevVectorStringStore = ["99 ramp 6 92",
                        "99 ramp 15 85"
                        ]

dqVectorStringStore = ["0 ramp 7 11",
                       "0 ramp 12 14",
                       "0 ramp 14 12",
                       "0 ramp 8 16 16 ramp 6 13",
                       ]

assetContainer = []

for i in range(0,5000):
    notional = np.random.choice(notionalStore)
    term = np.random.choice(termStore)
    intRate = np.random.choice(intRateStore)
    cdrVectorString = np.random.choice(cdrVectorStringStore)
    cprVectorString = np.random.choice(cprVectorStringStore)
    sevVectorString = np.random.choice(sevVectorStringStore)
    dqVectorString = np.random.choice(dqVectorStringStore)
    assetKwargs = SPCFUtils.convertToDict(    
    assetType = "Amortization",
    notional = notional, term = term,intRate = 0.2175,
    cdrVector = SPCFUtils.convertIntexRamp(cdrVectorString, term = term, divisor = 100),
    cprVector = SPCFUtils.convertIntexRamp(cprVectorString, term = term, divisor = 100),
    sevVector = SPCFUtils.convertIntexRamp(sevVectorString, term = term, divisor = 100),
    dqVector = SPCFUtils.convertIntexRamp(dqVectorString, term = term, divisor = 100),
    servicingFeesRatio = 0.005)
    assetContainer.append(Asset(**assetKwargs))
    

In [117]:
portfolioNotional = sum([asset.notional for asset in assetContainer])
portfolioWAC = sum([asset.notional * asset.intRate for asset in assetContainer]) / portfolioNotional
portfolioTerm = sum([asset.notional * asset.term for asset in assetContainer]) / portfolioNotional

aggregateCashflow = pd.concat([asset.cashflow for asset in assetContainer])
aggregateCashflow = aggregateCashflow.groupby('period').sum().reset_index()

aggregateCashflow.loc[:, "periodYears"] = np.floor_divide(aggregateCashflow.loc[:, "period"]-1, 12) + 1
aggregateCashflow["cumulativeInvestmentCash"] = aggregateCashflow["investmentCash"].cumsum()

aggregateCashflow["cumulativeDefaultPrin"] = aggregateCashflow["defaultPrin"].cumsum()
aggregateCashflow["cumulativeLossPrin"] = aggregateCashflow["lossPrin"].cumsum()

aggregateCashflow["cgl"] = aggregateCashflow["cumulativeDefaultPrin"] / portfolioNotional
aggregateCashflow["cnl"] = aggregateCashflow["cumulativeLossPrin"] / portfolioNotional

aggregateCashflow["balFactor"] = aggregateCashflow["eopBal"] / portfolioNotional

aggregateCashflow["dqVector"] = aggregateCashflow["dqBal"] / aggregateCashflow["bopBal"]

In [118]:
aggregateCashflow = aggregateCashflow[['period',
                                       'periodYears',
                                       'dqVector',
                                       'balFactor',
                                       'bopBal',
                                       'prinCF',
                                       'eopBal',
                                       'totalCF',
                                       'cumulativeDefaultPrin',
                                       'cumulativeLossPrin',
                                       'cgl',
                                       'cnl'
                                       ]]

# Hold to Maturity

In [119]:
wal = sum(aggregateCashflow["prinCF"] * aggregateCashflow["period"]) / sum(aggregateCashflow["prinCF"]) / 12.0
cgl = aggregateCashflow["cgl"].iloc[-1]
cnl = aggregateCashflow["cnl"].iloc[-1]
avgDq = sum(aggregateCashflow.iloc[1:]["dqVector"] * aggregateCashflow.iloc[1:]["eopBal"]) / sum(aggregateCashflow.iloc[1:]["eopBal"])
pnl = sum(aggregateCashflow["totalCF"]) - portfolioNotional
moic = 1.0 + pnl / portfolioNotional
servicing = 0.005
print(
      '\n',
      'portfolioNotional: ', portfolioNotional, '\n',
      'Term: ', portfolioTerm, '\n',
      'WAC: ', portfolioWAC, '\n',
      'WAL: ', wal, '\n',
      "CGL: ", cgl, '\n',
      "CNL: ", cnl, '\n',
      "Servicing: ", servicing, '\n',
      "Avg DQ: ", avgDq,'\n',
      "PnL (assuming Par): ", pnl , '\n',
      "MOIC (assuming Par): ", moic,  '\n',
)


 portfolioNotional:  137079700 
 Term:  56.20100423330369 
 WAC:  0.2175 
 WAL:  2.0721605360414928 
 CGL:  0.24988957959546035 
 CNL:  0.22377341163482226 
 Servicing:  0.005 
 Avg DQ:  0.10052524494650411 
 PnL (assuming Par):  21684955.650266916 
 MOIC (assuming Par):  1.1581923191418344 



In [148]:
df = pd.DataFrame(columns = ["purchasePx", "irr", "proxyIrr"])
for purchasePx in range(90, 110, 1):
    aggregateCashflow["purchaseCash"] = np.concatenate([np.array([portfolioNotional * purchasePx / 100.0]), np.array([0] * (aggregateCashflow.shape[0] - 1))])
    aggregateCashflow["investmentCash"] = - aggregateCashflow["purchaseCash"] + aggregateCashflow["totalCF"]
    # further from par, the proxy is less accurate
    df.loc[len(df)] = [purchasePx, 
                       npf.irr(aggregateCashflow["investmentCash"].values) * 12,
                       (portfolioWAC * (1 - avgDq) - servicing - cnl / wal) + (100 - purchasePx) / 100.0]
print(df)

    purchasePx       irr  proxyIrr
0         90.0  0.145114  0.182645
1         91.0  0.138256  0.172645
2         92.0  0.131521  0.162645
3         93.0  0.124907  0.152645
4         94.0  0.118410  0.142645
5         95.0  0.112026  0.132645
6         96.0  0.105751  0.122645
7         97.0  0.099583  0.112645
8         98.0  0.093519  0.102645
9         99.0  0.087555  0.092645
10       100.0  0.081688  0.082645
11       101.0  0.075917  0.072645
12       102.0  0.070238  0.062645
13       103.0  0.064649  0.052645
14       104.0  0.059147  0.042645
15       105.0  0.053730  0.032645
16       106.0  0.048397  0.022645
17       107.0  0.043143  0.012645
18       108.0  0.037969  0.002645
19       109.0  0.032870 -0.007355


# Hold and Sell

In [203]:
purchasePx = 100
sellPx = 98
sellPeriod = 25

aggregateCashflow["purchaseCash"] = 0
aggregateCashflow["purchaseCash"] = np.concatenate([np.array([portfolioNotional * purchasePx / 100.0]), np.array([0] * (aggregateCashflow.shape[0] - 1))])
aggregateCashflow["sellCash"] = np.concatenate([np.array([0] * (sellPeriod)), np.array([aggregateCashflow.loc[sellPeriod, 'eopBal'] * sellPx / 100.0]), np.array([0] * (aggregateCashflow.shape[0] - sellPeriod-1))])
holdSellCashflow = aggregateCashflow.loc[:25,:]
holdSellCashflow.loc[:, "allPrinCF"] = holdSellCashflow["prinCF"] + holdSellCashflow["sellCash"]

In [205]:

wal = sum(holdSellCashflow["allPrinCF"] * holdSellCashflow["period"]) / sum(holdSellCashflow["allPrinCF"]) / 12.0
cgl = holdSellCashflow["cgl"].iloc[-1]
cnl = holdSellCashflow["cnl"].iloc[-1]
avgDq = sum(holdSellCashflow.iloc[1:]["dqVector"] * holdSellCashflow.iloc[1:]["eopBal"]) / sum(holdSellCashflow.iloc[1:]["eopBal"])
pnl = sum(holdSellCashflow["totalCF"]) + sum(holdSellCashflow["sellCash"]) - sum(holdSellCashflow["purchaseCash"])
moic = 1.0 + pnl / sum(holdSellCashflow["purchaseCash"])
servicing = 0.005
print(
      '\n',
      'portfolioNotional: ', portfolioNotional, '\n',
      "PurchasePx: ", purchasePx, '\n',
      "sellPx: ", sellPx, '\n',
      "sellPeriod: ", sellPeriod, '\n',
      'Term: ', portfolioTerm, '\n',
      'WAC: ', portfolioWAC, '\n',
      'WAL: ', wal, '\n',
      "CGL: ", cgl, '\n',
      "CNL: ", cnl, '\n',
      "Servicing: ", servicing, '\n',
      "Avg DQ: ", avgDq,'\n',
      "PnL: ", pnl , '\n',
      "MOIC: ", moic,  '\n',
)


 portfolioNotional:  137079700 
 PurchasePx:  100 
 sellPx:  98 
 sellPeriod:  25 
 Term:  56.20100423330369 
 WAC:  0.2175 
 WAL:  1.5371232323019868 
 CGL:  0.1608218544560784 
 CNL:  0.1449024821623062 
 Servicing:  0.005 
 Avg DQ:  0.09251786546861768 
 PnL:  17880674.217812717 
 MOIC:  1.1304399865028354 



In [207]:
holdSellCashflow["investmentCash"] = - aggregateCashflow["purchaseCash"] + aggregateCashflow["totalCF"] + holdSellCashflow["sellCash"]

print("irr: ", npf.irr(holdSellCashflow["investmentCash"].values) * 12)

print("proxy irr: ", (portfolioWAC * (1 - avgDq) - servicing - cnl / wal) \
    - (100 - sellPx) / 100.0  * holdSellCashflow["balFactor"].iloc[-1] / wal)

irr:  0.09068920419881898
proxy irr:  0.0927427568151302
